# 0. Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook
import classifiers
import importlib
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

# 1. Data Load

In [ ]:
reactome_labels_df = pd.read_csv('../../data/processed/reactome_labels_apid_huri.csv', header = None)
reactome_labels = reactome_labels_df.transpose().to_numpy(dtype='int')
reactome_labels_df.index= pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0).index
reactome_labels_df.columns= pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0).columns

In [ ]:
reactome_hypergeometric = np.array(pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0).transpose())
reactome_closeness = np.array(pd.read_csv("../../data/processed/metrics/process_closeness_apid_huri.csv", sep=',', header=0, index_col=0).transpose())
reactome_betweenness = np.array(pd.read_csv("../../data/processed/metrics/process_betweenness_apid_huri.csv", sep=',', header=0, index_col=0).transpose())
reactome_rwr = np.array(pd.read_csv("../../data/processed/metrics/process_rwr_apid_huri.csv", sep=',', header=0, index_col=0).transpose())
reactome_fraction_betweenness = np.array(pd.read_csv("../../data/processed/metrics/process_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0).transpose())

reactome_hypergeometric_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

reactome_hypergeometric_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

In [ ]:
reactome_hypergeometric_fs_ar = pd.read_csv("../../data/processed/fs/reactome_hyper_fs_apid_huri.csv", sep=',', header=0)
reactome_closeness_fs_ar = pd.read_csv("../../data/processed/fs/reactome_closeness_fs_apid_huri.csv", sep=',', header=0)
reactome_betweenness_fs_ar = pd.read_csv("../../data/processed/fs/reactome_betweenness_fs_apid_huri.csv", sep=',', header=0)
reactome_rwr_fs_ar = pd.read_csv("../../data/processed/fs/reactome_rwr_fs_apid_huri.csv", sep=',', header=0)
reactome_fraction_betweenness_fs_ar = pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_fs_apid_huri.csv", sep=',', header=0)

reactome_hypergeometric_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_closeness_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_betweenness_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_rwr_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_fraction_betweenness_80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))

reactome_hypergeometric_protein80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_protein80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_closeness_protein80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_protein80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_betweenness_protein80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_protein80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_rwr_protein80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_protein80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))
reactome_fraction_betweenness_protein80_fs_ar = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_protein80_fs_apid_huri.csv", sep=',', header=0).transpose(),10, axis=1))

In [ ]:
reactome_hypergeometric_test_index = pd.read_csv("../../data/processed/fs/reactome_hyper_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_closeness_test_index = pd.read_csv("../../data/processed/fs/reactome_closeness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_betweenness_test_index = pd.read_csv("../../data/processed/fs/reactome_betweenness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_rwr_test_index = pd.read_csv("../../data/processed/fs/reactome_rwr_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_fraction_betweenness_test_index = pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1

reactome_hypergeometric_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
reactome_closeness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
reactome_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
reactome_rwr_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
reactome_fraction_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1

reactome_hypergeometric_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
reactome_closeness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
reactome_betweenness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
reactome_rwr_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
reactome_fraction_betweenness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1

In [ ]:
ppi80_hyper_list = []
ppi80_closeness_list = []
ppi80_betweenness_list = []
ppi80_fraction_betweenness_list = []
ppi80_rwr_list = []
label_list = []

for i, df in reactome_hypergeometric_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_hyper_list.append(df)
    label_list.append(reactome_labels_df[reactome_labels_df.index.isin(df.index)])

for i, df in reactome_closeness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_closeness_list.append(df)
    
for i, df in reactome_betweenness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_betweenness_list.append(df)
    
for i, df in reactome_rwr_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_rwr_list.append(df)
    
for i, df in reactome_fraction_betweenness_80.groupby(level=0):
    df.set_index('index', inplace=True)
    ppi80_fraction_betweenness_list.append(df)
    
protein80_hyper_list = []
protein80_closeness_list = []
protein80_betweenness_list = []
protein80_rwr_list = []
protein80_fraction_betweenness_list = []
protein_label_list = []

for i, df in reactome_hypergeometric_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_hyper_list.append(df)
    protein_label_list.append(reactome_labels_df[reactome_labels_df.index.isin(df.index)])
    
for i, df in reactome_closeness_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_closeness_list.append(df)
    
for i, df in reactome_betweenness_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_betweenness_list.append(df)

for i, df in reactome_rwr_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_rwr_list.append(df)
    
for i, df in reactome_fraction_betweenness_protein80.groupby(level=0):
    df.set_index('index', inplace=True)
    protein80_fraction_betweenness_list.append(df)

# 2. Classification Tasks

## 2.1. Complete Network

In [ ]:
importlib.reload(classifiers)

clf = LogisticRegression(random_state=22)

parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]

hyper_lgr_clf, hyper_lgr_proba_clf, hyper_cv, hyper_n_fs, hyper_models = classifiers.multiple_fs_classifier2(clf, parameters, reactome_hypergeometric, reactome_hypergeometric_test_index, reactome_hypergeometric_fs_ar, reactome_labels_df, jobs=2)
hyper_lgr_clf.to_csv('../../models/GAP-MINE/process/binary/hypergeometric_lgr.csv', index=False)
hyper_lgr_proba_clf.to_csv('../../models/GAP-MINE/process/probability/hypergeometric_lgr_proba.csv', index=False)
with open('../../models/GAP-MINE/process/cv_results/hypergeometric.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in hyper_cv))
with open("../../models/GAP-MINE/process/n_fs/hypergeometric.txt", "w") as f:
    for s in hyper_n_fs:
        f.write(str(s) +"\n")
with open("../../models/GAP-MINE/process/models/hyper.pckl", "wb") as f:
    for model in hyper_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
rwr_lgr_clf, rwr_lgr_proba_clf, rwr_cv, rwr_n_fs, rwr_models = classifiers.multiple_fs_classifier2(clf, parameters, reactome_rwr, reactome_rwr_test_index, reactome_rwr_fs_ar, reactome_labels_df, jobs=20)
rwr_lgr_clf.to_csv('../../models/GAP-MINE/process/binary/rwr_lgr.csv', index=False)
rwr_lgr_proba_clf.to_csv('../../models/GAP-MINE/process/probability/rwr_lgr_proba.csv', index=False)
with open('../../models/GAP-MINE/process/cv_results/rwr.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in rwr_cv))
with open("../../models/GAP-MINE/process/n_fs/rwr.txt", "w") as f:
    for s in rwr_n_fs:
        f.write(str(s) +"\n")
with open("../../models/GAP-MINE/process/models/rwr.pckl", "wb") as f:
    for model in rwr_models:
         pickle.dump(model, f)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
closeness_lgr_clf, closeness_lgr_proba_clf, closeness_cv, closeness_n_fs, closeness_models = classifiers.multiple_fs_classifier2(clf, parameters, reactome_closeness, reactome_closeness_test_index, reactome_closeness_fs_ar, reactome_labels, jobs=20)
closeness_lgr_clf.to_csv('../../models/GAP-MINE/disease/binary/closeness_lgr.csv', index=False)
closeness_lgr_proba_clf.to_csv('../../models/GAP-MINE/disease/probability/closeness_lgr_proba.csv', index=False)
with open('../../models/GAP-MINE/disease/cv_results/closeness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in closeness_cv))
with open("../../models/GAP-MINE/disease/n_fs/closeness.txt", "w") as f:
    for s in closeness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/GAP-MINE/process/models/closeness.pckl", "wb") as f:
    for model in closeness_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
betweenness_lgr_clf, betweenness_lgr_proba_clf, betweenness_cv, betweenness_n_fs, betweenness_models = classifiers.multiple_fs_classifier(clf, parameters, reactome_betweenness, reactome_betweenness_test_index, reactome_betweenness_fs_ar, reactome_labels_df, jobs=20)
betweenness_lgr_clf.to_csv('../../models/GAP-MINE/process/binary/betweenness_lgr.csv', index=False)
betweenness_lgr_proba_clf.to_csv('../../models/GAP-MINE/process/probability/betweenness_lgr_proba.csv', index=False)
with open('../../models/GAP-MINE/process/cv_results/betweenness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in betweenness_cv))
with open("../../models/GAP-MINE/process/n_fs/betweenness.txt", "w") as f:
    for s in betweenness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/GAP-MINE/process/models/betweenness.pckl", "wb") as f:
    for model in betweenness_models:
         pickle.dump(model, f)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
fraction_betweenness_lgr_clf, fraction_betweenness_lgr_proba_clf, fraction_betweenness_cv, fraction_betweenness_n_fs, fraction_betweenness_models = classifiers.multiple_fs_classifier(clf, parameters, reactome_fraction_betweenness, reactome_fraction_betweenness_test_index, reactome_fraction_betweenness_fs_ar, reactome_labels_df, jobs=20)
fraction_betweenness_lgr_clf.to_csv('../../models/GAP-MINE/process/binary/fraction_betweenness_lgr.csv', index=False)
fraction_betweenness_lgr_proba_clf.to_csv('../../models/GAP-MINE/process/probability/fraction_betweenness_lgr.csv', index=False)
with open('../../models/GAP-MINE/process/cv_results/fraction_betweenness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in fraction_betweenness_cv))
with open("../../models/GAP-MINE/process/n_fs/fraction_betweenness.txt", "w") as f:
    for s in fraction_betweenness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/GAP-MINE/process/models/fraction_betweenness.pckl", "wb") as f:
    for model in fraction_betweenness_models:
         pickle.dump(model, f)

## 2.2. Reduced Networks

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
hyper_80_lgr, hyper_80_lgr_proba, hyper_80_cv, hyper_80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, ppi80_hyper_list, reactome_hypergeometric_80_test_index, reactome_hypergeometric_80_fs_ar, label_list)
hyper_80_lgr.to_csv('../../models/GAP-MINE/process/binary/hypergeometric_80_lgr.csv', index=False)
hyper_80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/hypergeometric_80_lgr.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
hyper_protein80_lgr, hyper_protein80_lgr_proba, hyper_protein80_cv, hyper_protein80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, protein80_hyper_list, reactome_hypergeometric_protein80_test_index, reactome_hypergeometric_protein80_fs_ar, protein_label_list)
hyper_protein80_lgr.to_csv('../../models/GAP-MINE/process/binary/hypergeometric_protein80_lgr.csv', index=False)
hyper_protein80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/hypergeometric_protein80_lgr.csv', index=False)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
closeness_80_lgr, closeness_80_lgr_proba, closeness_80_cv, closeness_80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, ppi80_closeness_list, reactome_closeness_80_test_index, reactome_closeness_80_fs_ar, label_list)
closeness_80_lgr.to_csv('../../models/GAP-MINE/process/binary/closeness_80_lgr.csv', index=False)
closeness_80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/closeness_80_lgr.csv', index=False)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
closeness_protein80_lgr, closeness_protein80_lgr_proba, closeness_protein80_cv, closeness_protein80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, protein80_closeness_list, reactome_closeness_protein80_test_index, reactome_closeness_protein80_fs_ar, protein_label_list)
closeness_protein80_lgr.to_csv('../../models/GAP-MINE/process/binary/closeness_protein80_lgr.csv', index=False)
closeness_protein80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/closeness_protein80_lgr.csv', index=False)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
rwr_80_lgr, rwr_80_lgr_proba, rwr_80_cv, rwr_80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, ppi80_rwr_list, reactome_rwr_80_test_index, reactome_rwr_80_fs_ar, label_list)
rwr_80_lgr.to_csv('../../models/GAP-MINE/process/binary/rwr_80_lgr.csv', index=False)
rwr_80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/rwr_80_lgr.csv', index=False)

In [ ]:
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
rwr_protein80_lgr, rwr_protein80_lgr_proba, rwr_protein80_cv, rwr_protein80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, protein80_rwr_list, reactome_rwr_protein80_test_index, reactome_rwr_protein80_fs_ar, protein_label_list)
rwr_protein80_lgr.to_csv('../../models/GAP-MINE/process/binary/rwr_protein80_lgr.csv', index=False)
rwr_protein80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/rwr_protein80_lgr.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
fraction_betweenness_80_lgr, fraction_betweenness_80_lgr_proba, fraction_betweenness_80_cv, fraction_betweenness_80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, ppi80_fraction_betweenness_list, reactome_fraction_betweenness_80_test_index, reactome_fraction_betweenness_80_fs_ar, label_list)
fraction_betweenness_80_lgr.to_csv('../../models/GAP-MINE/process/binary/fraction_betweenness_80_lgr.csv', index=False)
fraction_betweenness_80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/fraction_betweenness_80_lgr.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
fraction_betweenness_protein80_lgr, fraction_betweenness_protein80_lgr_proba, fraction_betweenness_protein80_cv, fraction_betweenness_protein80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, protein80_fraction_betweenness_list, reactome_fraction_betweenness_protein80_test_index, reactome_fraction_betweenness_protein80_fs_ar, protein_label_list)
fraction_betweenness_protein80_lgr.to_csv('../../models/GAP-MINE/process/binary/fraction_betweenness_protein80_lgr.csv', index=False)
fraction_betweenness_protein80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/fraction_betweenness_protein80_lgr.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
betweenness_80_lgr, betweenness_80_lgr_proba, betweenness_80_cv, betweenness_80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, ppi80_betweenness_list, reactome_betweenness_80_test_index, reactome_betweenness_80_fs_ar, label_list)
betweenness_80_lgr.to_csv('../../models/GAP-MINE/process/binary/betweenness_80_lgr.csv', index=False)
betweenness_80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/betweenness_80_lgr.csv', index=False)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
betweenness_protein80_lgr, betweenness_protein80_lgr_proba, betweenness_protein80_cv, betweenness_protein80_n_fs = classifiers.reduced_classifier_multiple_fs(clf, parameters, protein80_betweenness_list, reactome_betweenness_protein80_test_index, reactome_betweenness_protein80_fs_ar, protein_label_list)
betweenness_protein80_lgr.to_csv('../../models/GAP-MINE/process/binary/betweenness_protein80_lgr.csv', index=False)
betweenness_protein80_lgr_proba.to_csv('../../models/GAP-MINE/process/probability/betweenness_protein80_lgr.csv', index=False)